In [1]:
import copy
import os
import time
import random
import sys
import warnings

import numpy as np
import pandas as pd
from typing import Any


from hyperimpute.plugins.imputers import Imputers
from hyperimpute.utils.distributions import enable_reproducible_results
from hyperimpute.utils.benchmarks import compare_models
import hyperimpute.logger as log

from IPython.display import HTML, display
import tabulate

import json

warnings.filterwarnings("ignore")
enable_reproducible_results()

imputers = Imputers()
log.add(sink=sys.stderr, level="INFO")

In [2]:
from pathlib import Path


def get_imputer():
    return imputers.get("hyperimpute", optimizer="bayesian")


def save_results(fname, results):
    path = Path("experiments_01_hyperimpute_with_bayesian_optimization")
    path.mkdir(parents=True, exist_ok=True)

    out = path / fname

    with open(out, "w") as outfile:
        json.dump(results, outfile)


def evaluate_dataset_repeated(
    name,
    X_raw,
    y,
    ref_methods=[
        "mean",
        "sklearn_missforest",
        "sklearn_ice",
        "gain",
        "sinkhorn",
        "softimpute",
    ],
    scenarios=["MNAR", "MCAR", "MAR"],
    miss_pct=[0.1, 0.3, 0.5, 0.7],
    n_iter=10,
):
    results = compare_models(
        name=name,
        evaluated_model=get_imputer(),
        X_raw=X_raw,
        ref_methods=ref_methods,
        scenarios=scenarios,
        miss_pct=miss_pct,
        n_iter=n_iter,
    )

    save_results(name, results)

## Sanity check in  debug mode

In [3]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat",
    header=None,
    sep="\\t",
)

y = df[5]
X_raw = df.drop(columns=[5])

evaluate_dataset_repeated(
    "airfoil_debug",
    X_raw,
    y,
    scenarios=["MNAR"],
    ref_methods=["mean", "miracle"],
    n_iter=1,
    miss_pct=[0.3],
)

[2024-03-11T06:53:23.202168+0800][23904][INFO] Iteration imputation: select_model_by_column: True, select_model_by_iteration: True
[2024-03-11T06:54:03.997161+0800][23904][INFO]   > HyperImpute using inner optimization
[2024-03-11T06:54:03.998159+0800][23904][INFO]   > Imputation iter 0
[2024-03-11T06:54:04.548606+0800][23904][INFO] [Evaluate 2] previous config new score = 0.4798893076971098. old score = -9999999
[2024-03-11T06:54:13.480351+0800][23904][INFO]      >>> Column 2 <-- score 0.7201745788014966 <-- Model xgboost_regressor({})
[2024-03-11T06:54:13.900348+0800][23904][INFO] [Evaluate 1] previous config new score = 0.6810132097042186. old score = -9999999
[2024-03-11T06:54:19.381023+0800][23904][INFO]      >>> Column 1 <-- score 0.7466934019095612 <-- Model xgboost_regressor({})
[2024-03-11T06:54:19.566025+0800][23904][INFO]   > Imputation iter 1
[2024-03-11T06:54:19.664023+0800][23904][INFO] [Evaluate 2] previous config new score = 0.9743061846665817. old score = 0.72017457880

Instructions for updating:
non-resource variables are not supported in the long term
RMSE score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,miracle
0,MNAR,0.3,0.2008 +/- 0.0,0.2913 +/- 0.0,0.2843 +/- 0.0




Wasserstein score


,Scenario,"miss_pct [0, 1]",Evaluated: hyperimpute,mean,miracle
0,MNAR,0.3,0.0943 +/- 0.0,0.3687 +/- 0.0,0.2493 +/- 0.0


# Datasets

In [4]:
import hyperimpute.logger as log

log.remove()

| Dataset     | Length | Features |
|-------------|--------|----------|
| airfoil     | 1503   | 6        |
| blood       | 748    | 5        |
| bc          | 569    | 30       |
| california  | 20640  | 8        |
| climate     | 540    | 21       |
| compression | 1030   | 9        |
| slump       | 103    | 11       |
| sonar       | 208    | 61       |
| diabetes    | 442    | 10       |
| wine_red    | 1599   | 12       |
| wine_white  | 4898   | 12       |
| yeast       | 1484   | 10       |
| iris        | 150    | 4        |
| libras      | 360    | 91       |
| parkinsons  | 195    | 24       |
| yacht       | 308    | 7        |
| ionosphere  | 351    | 35       |
| letter      | 20000  | 17       |
| spam        | 4600   | 58       |
| credit      | 690    | 16       |

## Dataset: UCI Airfoil Self-Noise Data Set

https://archive.ics.uci.edu/ml/datasets/airfoil+self-noise


In [5]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00291/airfoil_self_noise.dat",
    header=None,
    sep="\\t",
)

df

,0,1,2,3,4,5
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461
...,...,...,...,...,...,...
1498,2500,15.6,0.1016,39.6,0.052849,110.264
1499,3150,15.6,0.1016,39.6,0.052849,109.254
1500,4000,15.6,0.1016,39.6,0.052849,106.604
1501,5000,15.6,0.1016,39.6,0.052849,106.224


In [6]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("airfoil", X_raw, y)

## Dataset: UCI Blood Transfusion Service Center Data Set

https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data"
)

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("blood", X_raw, y)

## Dataset: Breast Cancer Wisconsin (Diagnostic)

https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

In [ ]:
from sklearn.datasets import load_breast_cancer

X_raw, y = load_breast_cancer(as_frame=True, return_X_y=True)

X_raw

In [ ]:
evaluate_dataset_repeated("bc", X_raw, y)

## Dataset: California Housing



In [ ]:
from sklearn.datasets import fetch_california_housing

X_raw, y = fetch_california_housing(as_frame=True, return_X_y=True)

X_raw

In [ ]:
evaluate_dataset_repeated("california", X_raw, y)

## Dataset: Climate Model Simulation Crashes
https://archive.ics.uci.edu/ml/datasets/climate+model+simulation+crashes

In [ ]:
samples = np.loadtxt(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00252/pop_failures.dat",
    skiprows=1,
)
df = pd.DataFrame(samples)

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("climate", X_raw, y)

## Concrete Compressive Strength Data Set
https://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength

In [ ]:
df = pd.read_excel(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls"
)

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("compression", X_raw, y)

## Concrete Slump Test Data Set

https://archive.ics.uci.edu/ml/datasets/concrete+slump+test

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data"
)

df

In [ ]:
last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=["No", last_col])

evaluate_dataset_repeated("slump", X_raw, y)

## Connectionist Bench (Sonar, Mines vs. Rocks) Data Set

https://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data",
    header=None,
)

df

In [ ]:
last_col = df.columns[-1]
y = (df[last_col] == "M").astype(int)
X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("sonar", X_raw, y)

## Wine-Red dataset

In [ ]:
# Wine Quality Data Set

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
    sep=";",
)

df

In [ ]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)

X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("wine_red", X_raw, y)

## Wine-White dataset

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv",
    sep=";",
)

df

In [ ]:
last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)

X_raw = df.drop(columns=[last_col])

evaluate_dataset_repeated("wine_white", X_raw, y)

## Diabetes


In [ ]:
from sklearn.datasets import load_diabetes

X, y = load_diabetes(as_frame=True, return_X_y=True)

X

In [ ]:
evaluate_dataset_repeated("diabetes", X, y)

## Iris


In [ ]:
from sklearn.datasets import load_iris

X, y = load_iris(as_frame=True, return_X_y=True)

X

In [ ]:
evaluate_dataset_repeated("iris", X, y)

## Ionosphere

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data",
    sep=",",
    header=None,
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("ionosphere", X_raw, y)

## Libras

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.data",
    sep=",",
    header=None,
)

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("libras", X_raw, y)

## Parkinsons

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data",
    sep=",",
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

df = df.drop(columns=["name"])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("parkinsons", X_raw, y)

## Yacht

In [ ]:
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00243/yacht_hydrodynamics.data",
    sep="\s+",
    header=None,
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("yacht", X_raw, y)

## Spam

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data"
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("spam", X_raw, y)

## Letter dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data",
    header=None,
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("letter", X_raw, y)

## Credit dataset

In [ ]:
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data",
    header=None,
)

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = LabelEncoder().fit_transform(df[col])

last_col = df.columns[-1]
y = df[last_col]
X_raw = df.drop(columns=[last_col])

X_raw

In [ ]:
evaluate_dataset_repeated("credit", X_raw, y)

In [ ]:
#